In [1]:
# from yandex_chain import YandexEmbeddings
from langchain_community.vectorstores import FAISS

import os

In [ ]:
# !pip install accelerate

In [ ]:
# !pip install -i https://pypi.org/simple/ bitsandbytes

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'IlyaGusev/saiga_llama3_8b'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # load_in_8bit=True,
    torch_dtype = torch.bfloat16,
    device_map = "auto"
)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [59]:
from transformers import GenerationConfig
generation_config = GenerationConfig.from_pretrained(model_id)
print(generation_config)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [61]:
type(generation_config)

transformers.generation.configuration_utils.GenerationConfig

In [137]:
generation_config.max_new_tokens = 300
generation_config.temperature = 0.2
generation_config.top_k= 10

In [138]:
def get_llm_answer(query, chunks_join):
    user_prompt = '''Используй только следующий контекст, чтобы очень кратко ответить на вопрос в конце.
    Не пытайся выдумывать ответ.
    Контекст:
    ===========
    {chunks_join}
    ===========
    Вопрос:
    ===========
    {query}'''.format(chunks_join=chunks_join, query=query)
    
    SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
    RESPONSE_TEMPLATE = "<|im_start|>assistant\n"
    
    prompt = f'''<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{user_prompt}<|im_end|>\n{RESPONSE_TEMPLATE}'''
    
    def generate(model, tokenizer, prompt):
        data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
        data = {k: v.to(model.device) for k, v in data.items()}
        output_ids = model.generate(
            **data,
            generation_config=generation_config
        )[0]
        output_ids = output_ids[len(data["input_ids"][0]) :]
        output = tokenizer.decode(output_ids, skip_special_tokens=True)
        return output.strip()
    
    response = generate(model, tokenizer, prompt)
    
    return response

### Поиск по описаниям вопросов

In [9]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")

/home/yagor/anaconda3/envs/torch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
# /home/yagor/Рабочий стол/atomic/AtomHack/data/unstructured_vdb
db2 = FAISS.load_local(
    os.path.join("..", "data", 'structured_vdb'),
    embeddings,
    allow_dangerous_deserialization = True
)

In [25]:
query = 'в счете не отображается номер договора и период оплаты.'#"нет графы для п суточных и количества дней проживания"
# docs = retriever.invoke("query") 'Почему документ создает переоценку только на последнее число месяца?' #'Есть несколько вопросов по формированию отчета Задолженность клиентов и поставщиков по срокам'

In [26]:
docs = db2.similarity_search_with_score(query, k=5)

In [29]:
docs[1]

(Document(page_content='В Карточке Договора отражены сведения по оплате: {cid:imageXXX.png@XXDAXXAC.XBXXXBFX] В заказе отражены сведения по датам и процентам оплаты: {cid:imageXXX.png@XXDAXXAD.XXCXAXXX] А при формировании отчета высвечивается просроченная задолженность: {cid:imageXXX.png@XXDAXXAE.DXXXACXX]', metadata={'Код': 'IM23362338', 'Категория': 'запрос на обслуживание', 'Время регистрации': '17.04.2024 10:18:03', 'Рабочая группа': '1с erp 2.0 1 линия - казначейство', 'Краткое описание': '1С 2.0 АО НИФХИ_ГК Росатом.', 'Решение': 'добрый день. Инструкции есть в рабочей базе в разделе «помощь пользователям и новости» пункт «инструкции и памятки». В приложении X к инструкции по формированию графиков описаны шаги по корректировке дат планового погашения.', 'Аналитика 1': 'Казначейство, взаиморасчеты', 'Аналитика 2': 'Взаиморасчеты', 'Аналитика 3': 'Графики отгрузки и оплаты'}),
 0.71577466)

In [30]:
norm_docs_from_excel = [i for i in docs if i[1]<=0.658243]

In [37]:
norm_docs_from_excel[0][0]

Document(page_content='Добрый день! При формировании счета на оплату клиентам в X С ЕРП X.X Цифровой Росатом, в счете не отображается номер договора и период оплаты.', metadata={'Код': 'IM22455038', 'Категория': 'запрос на обслуживание', 'Время регистрации': '17.01.2024 11:15:19', 'Рабочая группа': '1с erp 2.0 1-я линия', 'Краткое описание': '1 С ЕРП 2.0 Цифровой Росатом', 'Решение': 'Добрый день! Пользователь разобрался с функционалом самостоятельно', 'Аналитика 1': 'Опер.контур', 'Аналитика 2': 'Продажи', 'Аналитика 3': 'Прочее'})

In [139]:
if len(norm_docs_from_excel)>=0:
    real_question = norm_docs_from_excel[0][0]
    print('Обращение:' , real_question.page_content)
    print('Решение аналогичного вопроса:', real_question.metadata["Решение"])
    category = real_question.metadata["Аналитика 3"] if 'nan' != str(real_question.metadata['Аналитика 3']) else real_question.metadata["Аналитика 2"]
    if 'nan' == category:
        category = real_question.metadata["Аналитика 1"]
    print("Категория:", category)

Обращение: Добрый день! При формировании счета на оплату клиентам в X С ЕРП X.X Цифровой Росатом, в счете не отображается номер договора и период оплаты.
Решение аналогичного вопроса: Добрый день! Пользователь разобрался с функционалом самостоятельно
Категория: Прочее


### Поиск чанков с близкими эмбеддингами из векторной базы

In [40]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [41]:
embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")

/home/yagor/anaconda3/envs/torch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [43]:
# /home/yagor/Рабочий стол/atomic/AtomHack/data/unstructured_vdb
db = FAISS.load_local(
    os.path.join("..", "data", 'unstructured_vdb'),
    embeddings,
    allow_dangerous_deserialization = True
)

In [52]:
query = "Что является целью интеграционного сценария «Массовая выгрузка документов из ЕОСДО?"
docs = db.similarity_search(query, k=5)

In [53]:
docs

[Document(page_content='Автор АО «Гринатом» Инструкция пользователей «Интеграция 1С ERP 2.0: \nЦифровой Росатом с ЕОСДО » Страница 10 \nСтраниц 35 \nВерсия  6 \n1.2 Интеграционный сценарий «Массовая выгрузка доку ментов из \nЕОСДО» \nСхематично интеграционный сценарий изображён на Рис унке 1.2: \nРисунок 1.2. Сценарий «Массовая выгрузка документов  из ЕОСДО» \nФормирование \nреестра и отправка \nреестра в ЕОСДО \nПолучение и \nобработка реестра Выгрузка скан-\nобразов \nдокументов на \nфайловый сервер Подтверждение \nзавершения \nвыгрузки \nM .O.3 M .I.3\n \nЦелью интеграционного сценария «Массовая выгрузка д окументов из \nЕОСДО» является автоматизация предоставления реестр а скан-образов \nдокументов в контролирующие органы. \nЗависимые бизнес-процессы:  \n\uf0b7 D-1C1-1.10.19 Формирование налоговой декларации по НП; \n\uf0b7 D-1C1-1.10.15 Формирование налоговой декларации по НДС; \n\uf0b7 D-1C1-1.10.35 Учет средств целевого финансирования;  \n\uf0b7 D-1C1-1.10.40 Формирование бухгал

In [54]:
docs[0]

Document(page_content='Автор АО «Гринатом» Инструкция пользователей «Интеграция 1С ERP 2.0: \nЦифровой Росатом с ЕОСДО » Страница 10 \nСтраниц 35 \nВерсия  6 \n1.2 Интеграционный сценарий «Массовая выгрузка доку ментов из \nЕОСДО» \nСхематично интеграционный сценарий изображён на Рис унке 1.2: \nРисунок 1.2. Сценарий «Массовая выгрузка документов  из ЕОСДО» \nФормирование \nреестра и отправка \nреестра в ЕОСДО \nПолучение и \nобработка реестра Выгрузка скан-\nобразов \nдокументов на \nфайловый сервер Подтверждение \nзавершения \nвыгрузки \nM .O.3 M .I.3\n \nЦелью интеграционного сценария «Массовая выгрузка д окументов из \nЕОСДО» является автоматизация предоставления реестр а скан-образов \nдокументов в контролирующие органы. \nЗависимые бизнес-процессы:  \n\uf0b7 D-1C1-1.10.19 Формирование налоговой декларации по НП; \n\uf0b7 D-1C1-1.10.15 Формирование налоговой декларации по НДС; \n\uf0b7 D-1C1-1.10.35 Учет средств целевого финансирования;  \n\uf0b7 D-1C1-1.10.40 Формирование бухгалт

In [55]:
print(docs[0].page_content)
page = docs[0].metadata["source"].split("/")[-1].split("_")[-1].split(".")[0]
file_name = docs[0].metadata['source'].split('/')[-2] + ".pdf"
print(f"Страница: {page}", f"Исходный документ: {file_name}")

Автор АО «Гринатом» Инструкция пользователей «Интеграция 1С ERP 2.0: 
Цифровой Росатом с ЕОСДО » Страница 10 
Страниц 35 
Версия  6 
1.2 Интеграционный сценарий «Массовая выгрузка доку ментов из 
ЕОСДО» 
Схематично интеграционный сценарий изображён на Рис унке 1.2: 
Рисунок 1.2. Сценарий «Массовая выгрузка документов  из ЕОСДО» 
Формирование 
реестра и отправка 
реестра в ЕОСДО 
Получение и 
обработка реестра Выгрузка скан-
образов 
документов на 
файловый сервер Подтверждение 
завершения 
выгрузки 
M .O.3 M .I.3
 
Целью интеграционного сценария «Массовая выгрузка д окументов из 
ЕОСДО» является автоматизация предоставления реестр а скан-образов 
документов в контролирующие органы. 
Зависимые бизнес-процессы:  
 D-1C1-1.10.19 Формирование налоговой декларации по НП; 
 D-1C1-1.10.15 Формирование налоговой декларации по НДС; 
 D-1C1-1.10.35 Учет средств целевого финансирования;  
 D-1C1-1.10.40 Формирование бухгалтерской отчетности .
Страница: 10 Исходный документ: Инструкция Интегра

In [ ]:
# !pip install torch==2.2.2

In [ ]:
result = get_llm_answer(query, docs[0].page_content)

In [64]:
result

'Целью интеграционного сценария «Массовая выгрузка документов из ЕОСДО» является автоматизация предоставления реестра скан-образов документов в контролирующие органы. <|im_end|>  ```python\n```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  ```  

In [41]:
result

'Целью интеграционного сценария «Массовая выгрузка документов из ЕОСДО» является автоматизация предоставления реестра скан-образов документов в контролирующие органы. <|im_end|>'

In [65]:
query = """при настройке видов сертификатов.  Счета учета расчет ов с контрагентами, настраиваемые с использованием \nэлементов списка  Группы настроек финансового учета расчетов, представлены в \nвиде групп в соответствии с их применением в операциях:"""
docs = db2.similarity_search(query)

In [66]:
docs

[Document(page_content='при настройке видов сертификатов.  \n\uf0b7 Счета учета расчет ов с контрагентами, настраиваемые с использованием \nэлементов списка  Группы настроек финансового учета расчетов, представлены в \nвиде групп в соответствии с их применением в операциях:  \no Расчетов с клиентами;  \no Расчетов с поставщиками;  \no Расчетов с комиссионерами;  \no Расчетов с комитентами;  \no Расчетов с кредиторами;  \no Расчетов с дебиторами;  \no Расчетов с лизингодателями.  \n\uf0b7 Производства  – задаются счета учета прямых производственных затрат \n(номенклатурные затраты на счетах 20, 23, 29) для подразделений в рамках \nкаждой организац ии. \n\uf0b7 Внеоборотных активов  – счета учета внеоборотных активов, настраиваемые с \nиспользованием элементов списка  Группы финансового учета внеоборотных \nактивов  при ведении в информационной базе учета внеоборотных активов по \nверсии 2.4.  \n\uf0b7 Прочих расходов  – счета у чета постатейных расходов, настраиваемые с \nиспользованием

In [67]:
# большая температура -> придуманный ответ
result = get_llm_answer(query, docs[0].page_content)
print(result)

Счета учета расчетов с контрагентами настраиваются с использованием элементов списка "Группы настроек финансового учета расчетов" в виде групп в соответствии с их применением в операциях: Расчетов с клиентами; Расчетов с поставщиками; Расчетов с комиссионерами; Расчетов с комитентами; Расчетов с кредиторами; Расчетов с дебиторами; Расчетов с лизингодателями. <|im_end|>
```python
def main():
    print("Счета учета расчетов с контрагентами настраиваются с использованием элементов списка 'Группы настроек финансового учета расчетов' в виде групп в соответствии с их применением в операциях:")
    print("Расчетов с клиентами;")
    print("Расчетов с поставщиками;")
    print("Расчетов с комиссионерами;")
    print("Расчетов с комитентами;")
    print("Расчетов с кредиторами;")
    print("Расчетов с дебиторами;")
    print("Расчетов с лизингодателями.")

if __name__ == "__main__":
    main()
```

Этот код создает функцию `main`, которая выводит список групп, используемых для настройки счетов 

In [68]:
real_result = result.split('<|im_end|>')[0]

In [69]:
real_result

'Счета учета расчетов с контрагентами настраиваются с использованием элементов списка "Группы настроек финансового учета расчетов" в виде групп в соответствии с их применением в операциях: Расчетов с клиентами; Расчетов с поставщиками; Расчетов с комиссионерами; Расчетов с комитентами; Расчетов с кредиторами; Расчетов с дебиторами; Расчетов с лизингодателями. '

### reranker

In [74]:
docs[0]

Document(page_content='при настройке видов сертификатов.  \n\uf0b7 Счета учета расчет ов с контрагентами, настраиваемые с использованием \nэлементов списка  Группы настроек финансового учета расчетов, представлены в \nвиде групп в соответствии с их применением в операциях:  \no Расчетов с клиентами;  \no Расчетов с поставщиками;  \no Расчетов с комиссионерами;  \no Расчетов с комитентами;  \no Расчетов с кредиторами;  \no Расчетов с дебиторами;  \no Расчетов с лизингодателями.  \n\uf0b7 Производства  – задаются счета учета прямых производственных затрат \n(номенклатурные затраты на счетах 20, 23, 29) для подразделений в рамках \nкаждой организац ии. \n\uf0b7 Внеоборотных активов  – счета учета внеоборотных активов, настраиваемые с \nиспользованием элементов списка  Группы финансового учета внеоборотных \nактивов  при ведении в информационной базе учета внеоборотных активов по \nверсии 2.4.  \n\uf0b7 Прочих расходов  – счета у чета постатейных расходов, настраиваемые с \nиспользованием 

In [76]:
docs[0].page_content.split('. ')

['при настройке видов сертификатов',
 ' \n\uf0b7 Счета учета расчет ов с контрагентами, настраиваемые с использованием \nэлементов списка  Группы настроек финансового учета расчетов, представлены в \nвиде групп в соответствии с их применением в операциях:  \no Расчетов с клиентами;  \no Расчетов с поставщиками;  \no Расчетов с комиссионерами;  \no Расчетов с комитентами;  \no Расчетов с кредиторами;  \no Расчетов с дебиторами;  \no Расчетов с лизингодателями',
 ' \n\uf0b7 Производства  – задаются счета учета прямых производственных затрат \n(номенклатурные затраты на счетах 20, 23, 29) для подразделений в рамках \nкаждой организац ии',
 '\n\uf0b7 Внеоборотных активов  – счета учета внеоборотных активов, настраиваемые с \nиспользованием элементов списка  Группы финансового учета внеоборотных \nактивов  при ведении в информационной базе учета внеоборотных активов по \nверсии 2.4',
 ' \n\uf0b7 Прочих расходов  – счета у чета постатейных расходов, настраиваемые с \nиспользованием элементов

In [77]:
sentences = docs[0].page_content.split('. ')

In [141]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model_emb = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

tensor([[ 0.0355, -0.0222, -0.0135,  ..., -0.0340, -0.0576, -0.0533],
        [-0.0069, -0.0249, -0.0469,  ...,  0.0414, -0.0441, -0.0164],
        [ 0.0011, -0.0559, -0.0254,  ..., -0.0377, -0.0407, -0.0265],
        [-0.0212, -0.0323, -0.0088,  ...,  0.0140, -0.0365, -0.0433],
        [ 0.0332, -0.0448, -0.0358,  ...,  0.0015, -0.0095,  0.0010],
        [ 0.0335, -0.0338, -0.0176,  ...,  0.0246, -0.0365, -0.0460]])


In [100]:
def embeddings_text(sentences):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        model_output = model_emb(**encoded_input)
    embeddings = model_output.pooler_output
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings

In [97]:
docs

[Document(page_content='при настройке видов сертификатов.  \n\uf0b7 Счета учета расчет ов с контрагентами, настраиваемые с использованием \nэлементов списка  Группы настроек финансового учета расчетов, представлены в \nвиде групп в соответствии с их применением в операциях:  \no Расчетов с клиентами;  \no Расчетов с поставщиками;  \no Расчетов с комиссионерами;  \no Расчетов с комитентами;  \no Расчетов с кредиторами;  \no Расчетов с дебиторами;  \no Расчетов с лизингодателями.  \n\uf0b7 Производства  – задаются счета учета прямых производственных затрат \n(номенклатурные затраты на счетах 20, 23, 29) для подразделений в рамках \nкаждой организац ии. \n\uf0b7 Внеоборотных активов  – счета учета внеоборотных активов, настраиваемые с \nиспользованием элементов списка  Группы финансового учета внеоборотных \nактивов  при ведении в информационной базе учета внеоборотных активов по \nверсии 2.4.  \n\uf0b7 Прочих расходов  – счета у чета постатейных расходов, настраиваемые с \nиспользованием

In [107]:
def reranker_part_chunk(sentences, question_embedding):
    context_embeddings = embeddings_text(sentences)
    res = torch.nn.functional.cosine_similarity(context_embeddings, question_embedding, dim=1)
    return res

In [101]:
question_embedding = embeddings_text([query])

In [104]:
question_embedding.shape

torch.Size([1, 768])

In [ ]:
question_embedding

In [109]:
res = reranker_part_chunk(sentences, question_embedding)
res

tensor([0.5365, 0.8278, 0.4757, 0.6429, 0.6274])

In [130]:
def real_rerank(docs, query):
    question_embedding = embeddings_text([query])
    result_points = torch.zeros(len(docs))
    for i, doc in enumerate(docs):
        sentences = doc.page_content.split('. ')
        result_points[i] = reranker_part_chunk(sentences, question_embedding).max()
    return result_points

In [131]:
result_points=real_rerank(docs, query)

In [132]:
result_points

tensor([0.8278, 0.6588, 0.6250, 0.7323])

In [135]:
doc_real = docs[int(result_points.argmax())]

In [136]:
doc_real

Document(page_content='при настройке видов сертификатов.  \n\uf0b7 Счета учета расчет ов с контрагентами, настраиваемые с использованием \nэлементов списка  Группы настроек финансового учета расчетов, представлены в \nвиде групп в соответствии с их применением в операциях:  \no Расчетов с клиентами;  \no Расчетов с поставщиками;  \no Расчетов с комиссионерами;  \no Расчетов с комитентами;  \no Расчетов с кредиторами;  \no Расчетов с дебиторами;  \no Расчетов с лизингодателями.  \n\uf0b7 Производства  – задаются счета учета прямых производственных затрат \n(номенклатурные затраты на счетах 20, 23, 29) для подразделений в рамках \nкаждой организац ии. \n\uf0b7 Внеоборотных активов  – счета учета внеоборотных активов, настраиваемые с \nиспользованием элементов списка  Группы финансового учета внеоборотных \nактивов  при ведении в информационной базе учета внеоборотных активов по \nверсии 2.4.  \n\uf0b7 Прочих расходов  – счета у чета постатейных расходов, настраиваемые с \nиспользованием 